## Required Library Import

In [ ]:
import ee
from epistemx.ee_config import initialize_earth_engine
initialize_earth_engine()

Earth Engine initialization failed: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, or ee.Authenticate() in Python, and then retry.
Earth Engine initialization failed: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, or ee.Authenticate() in Python, and then retry.


False

In [ ]:
!python -m pip install ../epistemx --quiet
import geemap
from epistemx.module_1 import Reflectance_Data, Reflectance_Stats
from epistemx.helpers import get_aoi_from_gaul

## Image Collection Retrival

### Multispectral Bands

In [ ]:
# --- Area of Interest (AOI) ---
#Set the country and province for the AOI using GAUL admin boundaries
aoi = get_aoi_from_gaul(country="Indonesia", province="Sumatera Selatan")
#Alternatively, used geemap_shp_to_ee to directly used shapefile in your local machine
#Intialize the relfectance class data function
optical_reflectance = Reflectance_Data()
#define the start and end date for imagery collection
start = '2017-01-01'
end = '2017-12-31'
#get the image collection and corresponding statistics
landsat_data, meta = optical_reflectance.get_optical_data(aoi, start, end, optical_data='L8_SR', 
                                                           cloud_cover=40, compute_detailed_stats=False)
#visualization parameter
l8_sr_visparam = {
    'min': 0,
    'max': 0.4,
    'gamma': [0.95, 1.1, 1],
    'bands':['NIR', 'RED', 'GREEN']
}
#create mosaic between image collection, and clip based on AOI
mosaic_landsat = landsat_data.mosaic().clip(aoi)
#Alternatively you can use temporal aggregation (ee reducer) to create mode cloudless imagery
median_landsat = landsat_data.median().clip(aoi)
#Add the data to the map
Map = geemap.Map()
Map.addLayer(mosaic_landsat, l8_sr_visparam, 'L8 SR Mosaic')
Map.addLayer(median_landsat, l8_sr_visparam, 'L8 SR Median')
Map.addLayer(landsat_data, l8_sr_visparam, 'L8 SR Image Collection')
# set center of the map in the area of interest
Map.centerObject(aoi, 7)

### Thermal Band

In [ ]:
#retive thermal bands from TOA
thermal_bands, thermal_stats = optical_reflectance.get_thermal_bands(aoi, start, end, cloud_cover=40, compute_detailed_stats=False)
median_thermal = thermal_bands.median().clip(aoi)
thermal_vis = {
    'min': 286,
    'max': 300,
    'gammma': 0.4
}
#stacked all landsat bands
stacked_landsat = median_landsat.addBands(median_thermal)
#visualize the thermal bands and multispectral bands
Map.addLayer(median_thermal, thermal_vis, "Thermal Bands")
Map

## Image Retrival Report

In [ ]:
#intialize the statistic class
stats = Reflectance_Stats()
#get the retrival report and automatically print them
retrival_report = stats.get_collection_statistics(landsat_data, print_report=True)

In [ ]:
dwddw